
## 0. Install dependencies


In [ ]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])
print("✅ Installation complete")

In [ ]:
# Configure environment
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
print(f"✅ API Key: {os.getenv('GOOGLE_API_KEY', '')[:10]}...")

## 0.1 Configure Environment Variables

**Set up your Google API key and other configuration:**

# MomsHelperAI - Multi-Agent System Demo

**Google ADK Multi-Agent System for Indian Family Planning**

This notebook demonstrates a production-ready multi-agent system using:
- Google Agent Development Kit (ADK)
- Gemini 2.5 Flash Lite LLM
- SQLite for data persistence

## 1. Setup and Imports

In [ ]:
import asyncio
import os
from datetime import datetime, timedelta
import importlib
import sys

# Force reload modules to get latest changes
if 'agents.meal_planner' in sys.modules:
    importlib.reload(sys.modules['agents.base_agent'])
    importlib.reload(sys.modules['agents.search_agent'])
    importlib.reload(sys.modules['agents.meal_planner'])
    importlib.reload(sys.modules['agents.week_planner'])
    importlib.reload(sys.modules['agents.grocery_planner'])
    importlib.reload(sys.modules['agents.orchestrator'])
    print("🔄 Reloaded agent modules")

# MomsHelperAI imports
from agents.orchestrator import orchestrator
from agents.meal_planner import meal_planner_agent
from agents.week_planner import week_planner_agent
from agents.grocery_planner import grocery_planner_agent
from storage.sqlite_storage import SQLiteStorage
from utils.config import Config

print("✅ All imports successful")
print(f"✅ API Key configured: {Config.GOOGLE_API_KEY[:10]}...")
print(f"✅ Using Gemini model: gemini-2.5-flash-lite")

## 2. Initialize Storage and Load Sample Data

In [ ]:
# Initialize storage
storage = SQLiteStorage()

# Load sample Sharma family
sharma_family = {
    'id': 'sharma_001',
    'name': 'Sharma Family',
    'members': [
        {'name': 'Rajesh', 'age': 38, 'role': 'father'},
        {'name': 'Priya', 'age': 35, 'role': 'mother'},
        {'name': 'Aarav', 'age': 10, 'role': 'son'},
        {'name': 'Ananya', 'age': 7, 'role': 'daughter'}
    ],
    'dietary_restrictions': ['vegetarian'],
    'preferred_cuisines': ['North Indian', 'South Indian', 'Gujarati'],
    'allergies': [],
    'spice_level': 'medium'
}

try:
    storage.create_family(sharma_family)
    print("✅ Sharma family data loaded")
except:
    print("ℹ️  Sharma family already exists")

print(f"\n👨‍👩‍👧‍👦 Family: {sharma_family['name']}")
print(f"   Members: {len(sharma_family['members'])} (Rajesh, Priya, Aarav, Ananya)")
print(f"   Dietary: Vegetarian")
print(f"   Cuisines: {', '.join(sharma_family['preferred_cuisines'])}")

## 3. Demo 1: Meal Planning with MealPlannerAgent

**This demonstrates:**
- MealPlannerAgent using google_search for web recipes
- Real LLM response from Gemini

### ⚠️ Important: API Rate Limits

**Free tier limits for `gemini-2.5-flash-lite`:**
- 15 requests per minute
- Each meal plan uses ~5-10 API calls
- **Wait 60 seconds between runs if you get quota errors**

If you see "quota exceeded" error:
1. Wait 1 minute
2. Re-run the cell
3. Or use a different API key

In [ ]:
print("🍽️ Planning meals for the Sharma family...\n")
print("⏳ Calling MealPlanner Agent...")

try:
    response = await meal_planner_agent.plan_meals(
        family_id="sharma_001",
        request="Suggest easy and fast recepi for monday and tuesday, keep break on wednesday, thursday if not activity and have time then heavy meal.",
        num_days=7
    )
    
    print("="*70)
    print("🤖 MEAL PLAN RESPONSE:")
    print("="*70)
    
    # Extract the final text response from events
    if isinstance(response, list):
        # Find the last event with text content
        found_text = False
        for event in reversed(response):
            if hasattr(event, 'content') and event.content:
                # Check if parts exists and is not None
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            print(part.text)
                            found_text = True
                            break
                    if found_text:
                        break
        
        if not found_text:
            print("No text response found in events")
            print(f"\nTotal events: {len(response)}")
            # Show last few events for debugging
            if response:
                print("\nLast event details:")
                last_event = response[-1]
                print(f"  Event type: {type(last_event)}")
                if hasattr(last_event, 'content'):
                    print(f"  Has content: {last_event.content is not None}")
                    if last_event.content and hasattr(last_event.content, 'parts'):
                        print(f"  Parts: {last_event.content.parts}")
    elif hasattr(response, 'text'):
        print(response.text)
    else:
        print(response)
    
    print("\n" + "="*70)
    print("✅ Real AI-generated meal plan from Gemini!")
    print("="*70)
    
except Exception as e:
    error_msg = str(e)
    print(f"Error message: {error_msg}")
        

## 4. Demo 2: Natural Language Conversation

**Free-form chat with the orchestrator - Try your own request!**

In [ ]:
# ✏️ EDIT THIS - Try your own request!
my_request = "I need quick dinner ideas for tonight that kids will like"

print(f"Your request: {my_request}\n")
print("⏳ Calling MomsHelperAI orchestrator...\n")

response = await orchestrator.handle_request(
    user_request=my_request,
    family_id='sharma_001'
)

print("="*70)
print("🤖 AI Response:")
print("="*70)

if hasattr(response, 'text'):
    print(response.text)
else:
    print(response)

print("\n✅ Real-time AI response using Google ADK + Gemini!")

## 5. Architecture Summary

### 🏗️ Google ADK Multi-Agent Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    OrchestratorAgent                        │
│              (Root Coordinator - Gemini LLM)                │
│                                                             │
│  Uses LLM to decide which agents to call and when          │
└────────────────────┬────────────────────────────────────────┘
                     │
      ┌──────────────┼──────────────┬──────────────┐
      │              │              │              │
      ▼              ▼              ▼              ▼
┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐
│  Recipe  │  │   Meal   │  │   Week   │  │ Grocery  │
│ Refiner  │  │ Planner  │  │ Planner  │  │ Planner  │
│  Agent   │  │  Agent   │  │  Agent   │  │  Agent   │
└────┬─────┘  └────┬─────┘  └────┬─────┘  └────┬─────┘
     │             │             │             │
     │      ┌──────┘             │             │
     │      │ (uses as tool)     │             │
     │      ▼                    │             │
     │  ┌──────────┐             │             │
     │  │  Recipe  │             │             │
     │  │ Refiner  │             │             │
     │  └──────────┘             │             │
     │                           │             │
     ▼                           ▼             ▼
┌─────────┐               ┌──────────┐  ┌──────────┐
│ChromaDB │               │Activities│  │  Pantry  │
│ Recipes │               │   JSON   │  │  Tools   │
└─────────┘               └──────────┘  └──────────┘
```

### ✅ What Makes This REAL ADK:

1. **BaseAgent Class**: Wraps `google.adk.agents.Agent`
2. **Agent Instructions**: Each agent has specialized system prompts
3. **AgentTool Wrapping**: Sub-agents wrapped as tools for parent agents
4. **InMemoryRunner**: Proper ADK execution pattern
5. **FunctionTool**: Custom Python functions as agent tools
6. **Session Management**: Multi-turn conversations
7. **Real LLM Calls**: Every response from Gemini 2.0 Flash

### 🎯 Proof Points:

✅ **NOT hardcoded** - Every response is unique from Gemini LLM

✅ **Follows Kaggle examples** - Uses same patterns as day-1b and day-5a

✅ **Multi-agent coordination** - Orchestrator delegates to specialized agents

✅ **Tool integration** - ChromaDB, SQLite, custom functions as tools

✅ **Indian family context** - Cultural awareness in meal planning and activities

---

## 🎉 Conclusion

This notebook demonstrated **MomsHelperAI** - a production-ready multi-agent system using:

✅ **Google Agent Development Kit (ADK)**

✅ **Gemini 2.5 Flash Lite LLM** (REAL AI responses!)

✅ **Multi-agent architecture** with specialized sub-agents

✅ **SQLite** for data persistence

✅ **Indian family context** - culturally aware meal planning and activities

### Next Steps:

1. **Run CLI**: `python main.py` for interactive terminal interface
2. **Run API**: `python app.py` for REST API server
3. **Customize**: Add your own family data and preferences
4. **Extend**: Add more agents (fitness, budget, education)

---

**Built with ❤️ for Indian families using Google ADK**